In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

In [2]:
def convert_num_nam(string):
    "Given a string that is a concatenation of a jersey number and the name, return the two substring seperate"

    num = [letter for letter in string if letter.isdigit()]
    num = "".join(map(str, num))
    nam = [letter for letter in string if not letter.isdigit()]
    nam = "".join(map(str, nam))

    return num, nam
    
def get_field_df(soup):
    field_ratings = []
    field_players = []
    field_numbers = []
    for field_player in soup.find_all(class_=re.compile("LineupPlayerContainer")):
        rating_element = field_player.find_all(class_=re.compile("PlayerRatingStyled"))
        if len(rating_element) == 0:
            field_ratings.append("nan")
        else:
            field_ratings.append(rating_element[0].find_all("span")[0].get_text())
        
        player_text = field_player.find(class_=re.compile("LineupPlayerText"))
        num, name = convert_num_nam(player_text.get_text())
        field_players.append(name)
        field_numbers.append(num)

    return pd.DataFrame({"number": field_numbers, "name": field_players, "rating": field_ratings})
    
def get_bench_df(soup):
    bench_ratings = []
    bench_players = []
    bench_numbers = []
    for subsoup in soup.find_all(class_=re.compile("LeftBenchItem ")):  # the space is very important in the regex
        if 'PlayerRating' in str(subsoup):
            # extract name and number
            num, name = convert_num_nam((subsoup.find_all('span')[-2].get_text()))
            bench_players.append(name)
            bench_numbers.append(num)

            # get rating
            rating_element = subsoup.find(class_=re.compile("PlayerRatingStyled"))
            bench_ratings.append(rating_element.find('span').get_text())

    return pd.DataFrame({"number": bench_numbers, "name": bench_players, "rating": bench_ratings})

In [3]:
page = requests.get("https://www.fotmob.com/match/3602683/matchfacts/fc-utrecht-vs-go-ahead-eagles")
soup = BeautifulSoup(page.content, 'html.parser')



In [ ]:
bench_matchcard_urls = []

for bench_item in soup.find_all(class_=re.compile("LeftBenchItemOuter")):
    if 'PlayerRating' in str(bench_item):
        bench_matchcard_urls.append(f"https://www.fotmob.com{bench_item.find('a')['href']}")

In [4]:
bench_matchcard_urls

['https://www.fotmob.com/match/3602683/matchfacts/player-match-card/594983',
 'https://www.fotmob.com/match/3602683/matchfacts/player-match-card/304668',
 'https://www.fotmob.com/match/3602683/matchfacts/player-match-card/562846',
 'https://www.fotmob.com/match/3602683/matchfacts/player-match-card/902105',
 'https://www.fotmob.com/match/3602683/matchfacts/player-match-card/17761',
 'https://www.fotmob.com/match/3602683/matchfacts/player-match-card/957556',
 'https://www.fotmob.com/match/3602683/matchfacts/player-match-card/467480']

In [70]:
bmu_data = []
for bmu in bench_matchcard_urls:
    matchcard = requests.get(bmu)
    mc_soup = BeautifulSoup(matchcard.content, 'html.parser')
    name = mc_soup.find(class_=re.compile("jalymf-PlayerName")).get_text()
    bmu_data.append(mc_soup.find_all(class_=re.compile("RowContainer e1fnykti"))[1:])

In [28]:
columns = [row.find_all('span')[0].get_text() for row in bmu_data[0]]

In [75]:
bmu_data[0][1].find_all('span')

[<span>Minutes played</span>, <span>30</span>]

In [69]:
test = {'tim': [24], 'birgit': [24]}
test['rob'] = [25]
test

{'tim': [24], 'birgit': [24], 'rob': [25]}

In [64]:
matchcard_data = []
for bmud in bmu_data:
    for index, row in enumerate(bmud):
        spans = row.find_all('span')
        # check if we have data for column i
        if len(spans) < 2:
            data = 'Nan'
        else:
            data = spans[1].get_text()

        # first iteration, creating the nested list
        if len(matchcard_data) < 21:
            matchcard_data.append([data])
        # 2nd iteration and beyond
            
        else:
            print(index, data)
            matchcard_data[index].append(data)

    

matchcard_data

0 30
1 0
2 0
3 1
4 4/7 (57%)
5 0
6 0.03
7 0.03
8 Nan
9 1
10 12
11 0/1 (0%)
12 1
13 Nan
14 0
15 1
16 Nan
17 1/2 (50%)
18 3/4 (75%)
19 1
20 0
0 30
1 0
2 0
3 1
4 11/19 (58%)
5 0
6 0.05
7 Nan
8 1
9 32
10 1/1 (100%)
11 0/2 (0%)
12 0/4 (0%)
13 0
14 Nan
15 0
16 1
17 3
18 Nan
19 2/4 (50%)
20 3/4 (75%)
21 1


IndexError: list index out of range

In [42]:
for c, d in zip(columns, matchcard_data):
    print(c, d)

Minutes played 30
Goals 0
Assists 0
Total shots 2
Accurate passes 6/7 (86%)
Key passes 2
Expected goals (xG) 0.25
Expected assists (xA) 0.05
Attack NaN
Shot accuracy 0/2 (0%)
Big chance missed 1
Touches 15
Successful dribbles 1/2 (50%)
Dispossessed 2
Defence NaN
Tackles won 0
Duels NaN
Ground duels won 1/5 (20%)
Aerial duels won 1/1 (100%)
Was fouled 0
Fouls committed 1
